In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.color import rgb2lab, deltaE_cie76

# === Cấu hình thư mục ===
input_left = "D:/KLTN/SKINTONE/pre_processing/data4/output_crop_cheek_new/light/left"
input_right = "D:/KLTN/SKINTONE/pre_processing/data4/output_crop_cheek_new/light/right"
input_chin = "D:/KLTN/SKINTONE/pre_processing/data4/output_crop_chin_new/light"
output_combined = "D:/KLTN/SKINTONE/pre_processing/combined_new/light"
os.makedirs(output_combined, exist_ok=True)

# === Hàm tính màu trung bình trong không gian Lab ===
def get_mean_lab(img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    lab = rgb2lab(img_rgb)
    return lab.mean(axis=(0, 1))

# === Hàm tính khoảng cách DeltaE giữa 2 ảnh ===
def deltaE(img1, img2):
    lab1 = get_mean_lab(img1)
    lab2 = get_mean_lab(img2)
    return deltaE_cie76(lab1, lab2)

# === Bắt đầu xử lý ảnh ===
for f in tqdm(os.listdir(input_left), desc="Đang xử lý"):
    base_name = f.replace(".jpg", "")
    path_left = os.path.join(input_left, f)
    path_right = os.path.join(input_right, f)
    path_chin = os.path.join(input_chin, f)

    img_left = cv2.imread(path_left) if os.path.exists(path_left) else None
    img_right = cv2.imread(path_right) if os.path.exists(path_right) else None
    img_chin = cv2.imread(path_chin) if os.path.exists(path_chin) else None

    if img_left is None and img_right is None:
        print(f" Không có má nào: {base_name}")
        continue
    if img_chin is None:
        print(f" Thiếu ảnh cằm: {base_name}")
        continue

    # Resize tất cả về cùng chiều cao
    H = 112
    if img_left is not None:
        img_left = cv2.resize(img_left, (112, H))
    if img_right is not None:
        img_right = cv2.resize(img_right, (112, H))
    img_chin = cv2.resize(img_chin, (112, H))

    # Tính khoảng cách giữa các vùng
    parts_to_concat = []

    if img_left is not None and img_right is not None:
        delta_lr = deltaE(img_left, img_right)

        if delta_lr < 10:
            # Hai má gần nhau → kiểm tra cằm
            delta_lc = deltaE(img_left, img_chin)
            delta_rc = deltaE(img_right, img_chin)
            if delta_lc > 10 and delta_rc > 10:
                # Cằm lệch với cả 2 má → bỏ cằm
                parts_to_concat = [img_left, img_right]
            else:
                # Cằm phù hợp → giữ cả 3
                parts_to_concat = [img_left, img_chin, img_right]
        else:
            # Hai má lệch nhau → chọn má gần cằm
            delta_lc = deltaE(img_left, img_chin)
            delta_rc = deltaE(img_right, img_chin)

            if delta_lc < delta_rc:
                if delta_lc < 10:
                    parts_to_concat = [img_left, img_chin]
                else:
                    parts_to_concat = [img_left]
            else:
                if delta_rc < 10:
                    parts_to_concat = [img_chin, img_right]
                else:
                    parts_to_concat = [img_right]

    else:
        # Chỉ có 1 má
        if img_left is not None:
            delta_lc = deltaE(img_left, img_chin)
            if delta_lc < 10:
                parts_to_concat = [img_left, img_chin]
            else:
                parts_to_concat = [img_left]
        elif img_right is not None:
            delta_rc = deltaE(img_right, img_chin)
            if delta_rc < 10:
                parts_to_concat = [img_chin, img_right]
            else:
                parts_to_concat = [img_right]

    if len(parts_to_concat) < 2:
        print(f" Không đủ vùng hợp lệ để ghép: {base_name}")
        continue

    combined = cv2.hconcat(parts_to_concat)
    final_img = cv2.resize(combined, (224, 224))
    save_path = os.path.join(output_combined, base_name + "_filtered.jpg")
    cv2.imwrite(save_path, final_img)

print(" Đã gộp ảnh hoàn tất, loại bỏ cằm nếu lệch tone:", output_combined)


🔄 Đang xử lý:   0%|          | 31/6713 [00:01<04:17, 25.98it/s]

 Không đủ vùng hợp lệ để ghép: 100016175


🔄 Đang xử lý:   1%|          | 63/6713 [00:02<03:46, 29.37it/s]

 Không đủ vùng hợp lệ để ghép: 101405634


🔄 Đang xử lý:   1%|          | 69/6713 [00:02<04:00, 27.63it/s]

...

 Không đủ vùng hợp lệ để ghép: image_999


🔄 Đang xử lý: 100%|█████████▉| 6704/6713 [03:05<00:00, 55.58it/s]

 Không đủ vùng hợp lệ để ghép: z6661727880669_fb26afd4992f44dc2458dcad5473d24d


🔄 Đang xử lý: 100%|██████████| 6713/6713 [03:05<00:00, 36.22it/s]

 Không đủ vùng hợp lệ để ghép: z6661727977070_249bf39c62fb131f181419a6adc6fc5c
 Đã gộp ảnh hoàn tất, loại bỏ cằm nếu lệch tone: D:/KLTN/SKINTONE/pre_processing/combined_new/light
